In [14]:
import pandas as pd
import numpy as np
#import re
#import xlrd

# fileName - iterate through entire folder :)
fileName = 'Original_FiveAssetClasses.xlsx'
#fileNameOtherDevices = 'Other Device Numbers.xls'
#fileNamePolesClassHeight = 'Poles_class_height.xlsx' # Poles_class_height table

# Read xlsx file into dataframes
with pd.ExcelFile(fileName) as xlsx:
    #dfTopology = pd.read_excel(xlsx, 'Topology', index_col=None, na_values=['NA']) # IGNORE for now
    dfTransformersV1 = pd.read_excel(xlsx, 'Transformers') # 280 rows
    dfSwitchesV1 = pd.read_excel(xlsx, 'Switches') # Tot:239 - R/Y/B: 116/108/103 values; based on phases
    dfPolesV1 = pd.read_excel(xlsx, 'Poles') # 239 rows; 'Spot Number\n' col contains unique tx ids
    dfCablesV1 = pd.read_excel(xlsx, 'UGPrimaryCables')
    dfFusesV1 = pd.read_excel(xlsx, 'Fuses') # 44 items
    dfUGStructuresV1 = pd.read_excel(xlsx,'UGStructures')


Summary = {'Transformers:': dfTransformersV1.shape, 'Switches:': dfSwitchesV1.shape,'Poles:': dfPolesV1.shape, 
           'Cables:': dfCablesV1.shape, 'Fuses:':dfFusesV1.shape, 'UGStructures:':dfUGStructuresV1.shape}
dfSummary = pd.DataFrame(Summary)

# Make one copy
dfTransformersV2 = dfTransformersV1
dfSwitchesV2 = dfSwitchesV1
dfPolesV2 = dfPolesV1
dfCablesV2 = dfCablesV1
dfFusesV2 = dfFusesV1
dfUGStructuresV2 = dfUGStructuresV1

# 17 columns dropped
dropCommonColumns = ['OBJECTID','WORKORDERID','FIELDVERIFY','COMMENTS','CREATIONUSER','DATECREATED','LASTUSER',
                     'DATEMODIFIED','WORKREQUESTID','DESIGNID','WORKLOCATIONID','WMSID','WORKFLOWSTATUS',
                     'WORKFUNCTION','GISOTYPENBR','OWNERSHIP']

#*************************#
#****DEFINITIONS**********#
#*************************#
def drop_columns(dfAssetClass, dropColumns):
    dfAssetClass = dfAssetClass.drop(dropColumns, axis=1)
    return dfAssetClass

def new_columns(dfAssetClass, numAssetRows, columnID):
    dfAssetClass[columnID] = pd.DataFrame(np.empty([numAssetRows,1]).cumsum(axis=1))
    dfAssetClass.loc[:,columnID] = np.nan
    return dfAssetClass[columnID]
#*************************#
#****DEFINITIONS**********#
#*************************#

#Drop all common columns 
dfSwitchesV2 = drop_columns(dfSwitchesV2, dropCommonColumns)
dfTransformersV2 = drop_columns(dfTransformersV2, dropCommonColumns)
dfFusesV2 = drop_columns(dfFusesV2,dropCommonColumns)
dfCablesV2 = drop_columns(dfCablesV2, dropCommonColumns)
dfUGStructuresV2 = drop_columns(dfUGStructuresV2, dropCommonColumns)
dfPolesV2 = drop_columns(dfPolesV2, dropCommonColumns)

# Make one copy
dfTransformers = dfTransformersV2
dfSwitches = dfSwitchesV2
dfPoles = dfPolesV2
dfCables = dfCablesV2
dfFuses = dfFusesV2
dfUGStructures = dfUGStructuresV2

SummaryV2 = {'Transformers:': dfTransformers.shape, 'Switches:': dfSwitches.shape,'Poles:': dfPoles.shape, 
           'Cables:': dfCables.shape, 'Fuses:':dfFuses.shape, 'UGStructures:':dfUGStructures.shape}
dfSummaryV2 = pd.DataFrame(SummaryV2)
print(dfSummary)
print(dfSummaryV2)
# print(dfSummary-dfSummaryV2) # shows 17 cols dropped

   Cables:  Fuses:  Poles:  Switches:  Transformers:  UGStructures:
0     3865     736   18961        537           3618          16883
1       39      46      31         52             57             40
   Cables:  Fuses:  Poles:  Switches:  Transformers:  UGStructures:
0     3865     736   18961        537           3618          16883
1       23      30      15         36             41             24


In [17]:
# Save future wait times while running
dfTransformers = dfTransformersV2
dfSwitches = dfSwitchesV2
dfPoles = dfPolesV2
dfCables = dfCablesV2
dfFuses = dfFusesV2
dfUGStructures = dfUGStructuresV2

#print(dfPoles.columns)

dropTxCols = ['ANCILLARYROLE', 'ENABLED', 'FEEDERID2', 'FEEDERINFO', 'ELECTRICTRACEWEIGHT', 'LOCATIONID', 'SYMBOLROTATION', 
              'GPSDATE', 'LABELTEXT', 'OPERATINGVOLTAGE', 'NOMINALVOLTAGE', 'GROUNDREACTANCE', 'GROUNDRESISTANCE', 
              'MAGNETIZINGREACTANCE', 'MAGNETIZINGRESISTANCE', 'HIGHSIDEGROUNDREACTANCE','HIGHSIDEGROUNDRESISTANCE', 
              'HIGHSIDEPROTECTION', 'LOCATIONTYPE','COOLINGTYPE', 'FEATURE_STATUS','KVA', 'DEMAND_KVA',
              'DEMAND_DATE_MM_DD_YYYY', 'STREET_LIGHT_FACILITY', 'HIGHSIDECONFIGURATION', 'LOWSIDECONFIGURATION',
              'LOWSIDEGROUNDRESISTANCE', 'LOWSIDEVOLTAGE', 'LATITUDE', 'LONGITUDE']

dropPolesCols = ['SYMBOLROTATION','GPSDATE','SUBTYPECD','LABELTEXT','STRUCTURENUMBER','FEATURE_STATUS',
                 'STREETLIGHT_FACILITY','REPLACED_DATE_MM_DD_YYYY','CONDITION','CONDITION_STATUS','CONDITION_DATE']

# drop columns not related to Tx
dfTransformers = drop_columns(dfTransformers,dropTxCols)

# Add additional columns and fill with NaNs
numTxRows = len(dfTransformers['DEVICENUMBER'])

# keep only distribution poles
dfPoles = dfPoles[dfPoles.SUBTYPECD == 1] #1 - Dist(47%), 5-TrafficLights(2%), 7-streetlight(51%)

dropPolesCols = ['SYMBOLROTATION','GPSDATE','SUBTYPECD','LABELTEXT','STRUCTURENUMBER','FEATURE_STATUS',
                 'STREETLIGHT_FACILITY','REPLACED_DATE_MM_DD_YYYY','CONDITION','CONDITION_STATUS','CONDITION_DATE']

# drop columns and drop rows that are not UG Switch
dfPoles = drop_columns(dfPoles,dropPolesCols)


dfTransformers['Asset Class'] = new_columns(dfTransformers, numTxRows,'Asset Class')
dfTransformers['HI'] = new_columns(dfTransformers, numTxRows,'HI')
dfTransformers['PRID'] = new_columns(dfTransformers, numTxRows,'PRID')
dfTransformers['IN_VALLEY'] = new_columns(dfTransformers, numTxRows,'IN_VALLEY')
dfTransformers['TX_RESIDENTIAL'] = new_columns(dfTransformers, numTxRows,'TX_RESIDENTIAL')
dfTransformers['TX_COMMERCIAL'] = new_columns(dfTransformers, numTxRows,'TX_COMMERCIAL')
dfTransformers['TX_INDUSTRIAL'] = new_columns(dfTransformers, numTxRows,'TX_INDUSTRIAL')
dfTransformers['DEVICE_RESIDENTIAL'] = new_columns(dfTransformers, numTxRows,'DEVICE_RESIDENTIAL')
dfTransformers['DEVICE_COMMERCIAL'] = new_columns(dfTransformers, numTxRows,'DEVICE_COMMERCIAL')
dfTransformers['DEVICE_INDUSTRIAL'] = new_columns(dfTransformers, numTxRows,'DEVICE_INDUSTRIAL')
dfTransformers['UPSTREAM_DEVICE'] = new_columns(dfTransformers, numTxRows,'UPSTREAM_DEVICE')
dfTransformers['PCB'] = new_columns(dfTransformers, numTxRows,'PCB')

# Rename Pole columns
dfTransformers = dfTransformers.rename(columns={'DEVICENUMBER':'ID',
                                                'PHASEDESIGNATION':'Type',
                                                'INSTALLATIONDATE':'INSTALL_YEAR',
                                                'FEEDERID':'CIRCUIT',
                                                'RATEDKVA':'KVA'})
# Separate year
dfTransformers['INSTALL_YEAR'] = dfTransformers['INSTALL_YEAR'].apply(lambda x: x.year)

# UG Tx: 2/3/5/7 - 1Ph/Ntwk/Sub/Pad 3Ph [1436,27,4,507: 1642 counts]
# To avoid index vs copy error: pd.DataFrame...necessary (spent 4 hours getting rid of the warning error!)
# OH Tx: 1/9/10 - 1Ph/3Ph/2Ph [1125/510/7: 1347 counts]
dfOHTransformers = pd.DataFrame(dfTransformers[(dfTransformers.SUBTYPECD == 1) | 
                                  (dfTransformers.SUBTYPECD == 9) | 
                                  (dfTransformers.SUBTYPECD == 10)])

# Replace Asset class and 'SUBTYPECD' with actual tx types
dictOHTxSubclass = {'1':'Standard 1Ph','9':'Standard 3Ph','10':'Standard 2Ph'}

numOHTxRows = len(dfOHTransformers['ID'])

#dfOHTransformers['SUBTYPECD_LOOKUP'] = new_columns(dfOHTransformers, numOHTxRows,'SUBTYPECD_LOOKUP')
# pd.DataFrame(np.empty([numOHTxRows,1]).cumsum(axis=1))
#print(dfOHTransformers.head())

dfOHTransformers['SUBTYPECD'] = dfOHTransformers['SUBTYPECD'].astype(str)

#Try using .loc[row_indexer,col_indexer] = value instead

dfOHTransformers.loc[:,'SUBTYPECD'] = dfOHTransformers['SUBTYPECD'].apply(lambda x: dictOHTxSubclass[x])

# Fill in Asset and asset subclass columns
dfOHTransformers['Asset Class'] = 'Overhead Transformer'
dfOHTransformers = dfOHTransformers.rename(columns={'SUBTYPECD':'Asset Subclass'})

# Remaining OH Tx and UG Tx specific columns
dfOHTransformers['BANKING'] = new_columns(dfOHTransformers, numOHTxRows,'BANKING')

# Tx Domain code tables
fileNameDomainCodes_Tx = 'DomainCodes_Tx.xlsx'
# Read Other Device Numbers into dataframes
with pd.ExcelFile(fileNameDomainCodes_Tx) as xls:
    #dfTopology = pd.read_excel(xlsx, 'Topology', index_col=None, na_values=['NA']) # IGNORE for now
    dfUGTxDomainCodes = pd.read_excel(xls, 'UGTransformers')
    dfOHTxDomainCodes = pd.read_excel(xls, 'OHTransformers')

# Convert to string for merge purposes
dfOHTransformers['COMPATIBLEUNITID'] = dfOHTransformers['COMPATIBLEUNITID'].astype(str)
dfOHTxDomainCodes['COMPATIBLEUNITID'] = dfOHTxDomainCodes['COMPATIBLEUNITID'].astype(str)
#print(dfUGTxDomainCodes.head())

#dfOHTransformers=pd.merge(dfOHTransformers, dfOHTxDomainCodes, how='left', on='COMPATIBLEUNITID')
dfOHTransformers=dfOHTransformers.merge(dfOHTxDomainCodes, how='left', on='COMPATIBLEUNITID')
dropTxCols2 = ['COMPATIBLEUNITID','Description']

# drop columns and drop rows that are not UG Switch
dfOHTransformers = drop_columns(dfOHTransformers,dropTxCols2)

# Add additional columns and fill with NaNs
#dfPoles[''] = new_columns(dfPoles, numPolesRows,'')
numPolesRows = len(dfPoles['DEVICENUMBER'])
dfPoles['Asset Class'] = new_columns(dfPoles, numPolesRows,'Asset Class')
dfPoles['Asset Subclass'] = new_columns(dfPoles, numPolesRows,'Asset Subclass')
dfPoles['HI'] = new_columns(dfPoles, numPolesRows,'HI')
dfPoles['PHASING'] = new_columns(dfPoles, numPolesRows,'PHASING')
dfPoles['PRID'] = new_columns(dfPoles, numPolesRows,'PRID')
dfPoles['TX'] = new_columns(dfPoles, numPolesRows,'TX')
dfPoles['TX_TYPE'] = new_columns(dfPoles, numPolesRows,'TX_TYPE')
dfPoles['CIRCUIT1'] = new_columns(dfPoles, numPolesRows,'CIRCUIT1')
dfPoles['CIRCUIT2'] = new_columns(dfPoles, numPolesRows,'CIRCUIT2')
dfPoles['CIRCUIT3'] = new_columns(dfPoles, numPolesRows,'CIRCUIT3')
dfPoles['CIRCUIT4'] = new_columns(dfPoles, numPolesRows,'CIRCUIT4')
dfPoles['CIRCUIT5'] = new_columns(dfPoles, numPolesRows,'CIRCUIT5')
dfPoles['CIRCUIT6'] = new_columns(dfPoles, numPolesRows,'CIRCUIT6')
dfPoles['CIRCUIT7'] = new_columns(dfPoles, numPolesRows,'CIRCUIT7')
dfPoles['CIRCUIT8'] = new_columns(dfPoles, numPolesRows,'CIRCUIT8')
dfPoles['IN_VALLEY'] = new_columns(dfPoles, numPolesRows,'IN_VALLEY')
dfPoles['TX_RESIDENTIAL'] = new_columns(dfPoles, numPolesRows,'TX_RESIDENTIAL')
dfPoles['TX_COMMERCIAL'] = new_columns(dfPoles, numPolesRows,'TX_COMMERCIAL')
dfPoles['TX_INDUSTRIAL'] = new_columns(dfPoles, numPolesRows,'TX_INDUSTRIAL')
dfPoles['HEIGHT'] = new_columns(dfPoles, numPolesRows,'HEIGHT')
dfPoles['NUM_CIRCUITS'] = new_columns(dfPoles, numPolesRows,'NUM_CIRCUITS')
dfPoles['DEVICE'] = new_columns(dfPoles, numPolesRows,'DEVICE')
dfPoles['TX_KVA'] = new_columns(dfPoles, numPolesRows,'TX_KVA')
dfPoles['TX_PHASING'] = new_columns(dfPoles, numPolesRows,'TX_PHASING')

# Fill with values
dfPoles['Asset Class'] = 'Pole'
dfPoles['Asset Subclass'] = 'Wood'

# Poles_class_height table
fileNamePolesClassHeight = 'DomainCodes_PolesClassHeight.xlsx'
# Read Other Device Numbers into dataframes
with pd.ExcelFile(fileNamePolesClassHeight) as xls:
    #dfTopology = pd.read_excel(xlsx, 'Topology', index_col=None, na_values=['NA']) # IGNORE for now
    dfPolesClassHeight = pd.read_excel(xls, 'Sheet1') # 280 rows

#print(dfPoles.shape) # (8826, 27) 
#print(dfPolesClassHeight.shape)
#print(dfPoles.columns)
#print(dfPoles.dtypes)
#print(dfPolesClassHeight.dtypes)
#print('dfSG Shape:', dfSwitchGears.shape) #(111, 3)
#print(dfPoles['GISONUMBER'].describe()) #count:8619, max:1104737, min:34
print('**************')
#print(dfTransformers['GISONUMBER'].describe())#count:3435, max:1106210, min:34


**************


In [18]:
# Merge tables
dfPoles = dfPoles.merge(dfPolesClassHeight, how='left', on='COMPATIBLEUNITID')

# Rename Pole columns
dfPoles = dfPoles.rename(columns={'DEVICENUMBER':'ID','COMPATIBLEUNITID':'Type','INSTALLATIONDATE':'INSTALL_YEAR'})
# Separate year
dfPoles['INSTALL_YEAR'] = dfPoles['INSTALL_YEAR'].apply(lambda x: x.year)
#print(dfPoles.head(2))

#NEED to Rearrange columns - will do when pole attachment info available
#dfPoles=dfSwitches[['Asset Class', 'Asset Subclass', 'ID','CIRCUIT','INSTALL_YEAR','TIE_FEEDER','PHASING','IN_VALLEY','HI','TX_PHASE','PRID','Type']]

dfPoles = dfPoles.merge(dfOHTransformers, how='left', on='GISONUMBER')

MasterFile = pd.ExcelWriter('V2_GIS_PolesTx.xlsx')
dfPoles.to_excel(MasterFile, 'Sheet1')
MasterFile.save()

In [97]:
#***********************
# pole attached assets
#***********************

fileNameOtherDevices = 'Other Device Numbers.xlsx'
# Read Other Device Numbers into dataframes
with pd.ExcelFile(fileNameOtherDevices) as xls:
    #dfTopology = pd.read_excel(xlsx, 'Topology', index_col=None, na_values=['NA']) # IGNORE for now
    dfSwitchGears = pd.read_excel(xls, 'SWITCHGEARS') # 280 rows

dropSGcols = ['Switch Gear', 'Adrs #','Location','City','Notes','To Type','Inst. Date','Mftr.','Catalog#','Serial#',
             'DOM','Comments']

dfSwitchGears = drop_columns(dfSwitchGears, dropSGcols)
#dfSwitchGears = dfSwitchGears.dropna() # drop all rows with NaN values

dictSGassetSubclass = {'PMH-3':'Air-Insulated Live Front','PMH-5':'Air-Insulated Live Front',
                       'PMH-9':'Air-Insulated Live Front','PMH-11':'Air-Insulated Live Front',
                       'PME-9':'Air-Insulated Dead Front','PME-10':'Air-Insulated Dead Front',
                       'PME-11':'Air-Insulated Live Front','VISTA-321':'SF6-Insulated Switch',
                       'VISTA-422':'SF6-Insulated Switch','VISTA-431':'SF6-Insulated Switch',
                       '422':'S&C Elec','431':'S&C Elec','321':'S&C Elec','G&W':'G&W',
                       'NET':'Carte Elec Ltd'}
# 'Type' -> 'PMH'
# 'Loc_No' -> '149-S'
dfSwitchGears['Type'] = dfSwitchGears['Type'].fillna(method='ffill')
numSGrows = len(dfSwitchGears['Loc_No'])
dfSwitchGears['Subclass'] = new_columns(dfSwitchGears, numSGrows, 'Subclass')
dfSwitchGears =dfSwitchGears.astype(str)
dfSwitchGears['Subclass'] = dfSwitchGears['Type'].apply(lambda x: dictSGassetSubclass[x])

print(dfSwitchGears.head(4))

     Type Loc_No                  Subclass
0  PME-11  149-S  Air-Insulated Live Front
1  PME-11  165-S  Air-Insulated Live Front
2  PME-11  205-S  Air-Insulated Live Front
3  PME-11  269-F  Air-Insulated Live Front


In [99]:
# Replace 'Asset Subclass' col with actual names
dfSwitches['ID'] = dfSwitches['ID'].astype(str)
dfSwitches=pd.merge(dfSwitches, dfSwitchGears, how='left', left_on='ID', right_on='Loc_No')
#dfSwitches['ID'] = dfSwitchGears['Loc_No'].apply(lambda x: )
#df.merge(df1, on='sku', how='left')
# print(len(pd.unique(dfSwitchGears['Loc_No'].values.ravel()))) # 111

switchesDropMoreCols = ['Asset Subclass', 'Loc_No']
dfSwitches = dfSwitches.drop(switchesDropMoreCols, axis=1)
dfSwitches = dfSwitches.rename(columns={'Subclass':'Asset Subclass'})
#Rearrange columns
dfSwitches=dfSwitches[['Asset Class', 'Asset Subclass', 'ID','CIRCUIT','INSTALL_YEAR','TIE_FEEDER','PHASING','IN_VALLEY','HI','TX_PHASE','PRID','Type']]

MasterFile = pd.ExcelWriter('V2_GIS_Poles.xlsx')
dfSwitches.to_excel(MasterFile, 'Sheet1')
MasterFile.save()

In [ ]:
print(dfSwitches.shape)
print('dfSG Shape:', dfSwitchGears.shape)
print(dfSwitchGears.uniquevalues())
# print(dfSwitches.head(5))
# print(dfSwitchGears.head(10))

In [26]:
#SwitchGrouped = dfNodesCopy.groupby('SwitchRegion')
#print(SwitchGrouped.head(3))

In [8]:
# ****************************
# G. PRID to each region
# ****************************
# combine PRID to Tx?

# ****************************
# H. Cable 
# ****************************
# combine cable and conductors

# ****************************
# I. Conductors  
# ****************************
# combine poles

# ****************************
# J. Poles 
# ****************************
# Output excel file

